In [ ]:
!pip install langchain -q -U
!pip install langchain-community -q -U
!pip install -U langchain-community faiss-cpu tiktoken -q
!pip install pymupdf -q
!pip install TextLoader -q
!pip install CSVLoader -q
!pip install sentence_transformers -q
!pip install einops -q
!pip install rank_bm25 -q
!pip install flask
!pip install mesop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 43.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 55.1 MB/s e

In [ ]:
import os
import faiss
from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceEmbeddings
#from langchain_community.vectorstores.utils import DistanceStrategy
EMBEDDING_MODEL_NAME = "hkunlp/instructor-large"
embeddings = HuggingFaceEmbeddings(
    model_name = EMBEDDING_MODEL_NAME,
    model_kwargs = {"device": "cpu"},
    encode_kwargs = {"normalize_embeddings": False},  # Set `True` for cosine similarity
)

folder_path = '/content/test_folder'

# **Loading Docs**
Importing all the files in list called documents. Works on a if else statement, separate loaders used for different types of files, ex: pymupdf used for pdf.

In [ ]:
documents = []

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    print(f"Processing file: {file_path}")

    if filename.endswith('.pdf'):
        loader = PyMuPDFLoader(file_path)
        loaded_docs = loader.load()
        print(f"Loaded {len(loaded_docs)} documents from {filename}")

        for doc in loaded_docs:
            # Add metadata
            doc.metadata['source'] = filename
            print(f"Document: {doc.metadata['source']} - Page: {doc.metadata['page']}")
        documents.extend(loaded_docs)

    elif filename.endswith('.txt'):
        loader = TextLoader(file_path)
        loaded_docs = loader.load()
        print(f"Loaded {len(loaded_docs)} documents from {filename}")

        for doc in loaded_docs:
            # Add metadata
            doc.metadata['source'] = filename
            print(f"Document: {doc.metadata['source']} - Page: {doc.metadata['page']}")
        documents.extend(loaded_docs)

    elif filename.endswith('.csv'):
        loader = CSVLoader(file_path)
        loaded_docs = loader.load()
        print(f"Loaded {len(loaded_docs)} documents from {filename}")

        for doc in loaded_docs:
            # Add metadata
            doc.metadata['source'] = filename
            print(f"Document: {doc.metadata['source']} - Page: {doc.metadata['page']}")
        documents.extend(loaded_docs)

print(f"Total documents loaded: {len(documents)}")

Processing file: /content/test_folder/Bitcoin Whitepaper.pdf
Loaded 9 documents from Bitcoin Whitepaper.pdf
Document: Bitcoin Whitepaper.pdf - Page: 0
Document: Bitcoin Whitepaper.pdf - Page: 1
Document: Bitcoin Whitepaper.pdf - Page: 2
Document: Bitcoin Whitepaper.pdf - Page: 3
Document: Bitcoin Whitepaper.pdf - Page: 4
Document: Bitcoin Whitepaper.pdf - Page: 5
Document: Bitcoin Whitepaper.pdf - Page: 6
Document: Bitcoin Whitepaper.pdf - Page: 7
Document: Bitcoin Whitepaper.pdf - Page: 8
Processing file: /content/test_folder/Capital Budgeting[2867].pdf
Loaded 28 documents from Capital Budgeting[2867].pdf
Document: Capital Budgeting[2867].pdf - Page: 0
Document: Capital Budgeting[2867].pdf - Page: 1
Document: Capital Budgeting[2867].pdf - Page: 2
Document: Capital Budgeting[2867].pdf - Page: 3
Document: Capital Budgeting[2867].pdf - Page: 4
Document: Capital Budgeting[2867].pdf - Page: 5
Document: Capital Budgeting[2867].pdf - Page: 6
Document: Capital Budgeting[2867].pdf - Page: 7
Do

# **Testing**
Checking whether all the content from the folder is uploaded

In [ ]:
test_string = "Satoshi Nakamoto"
count = 0
for document in documents:
    if test_string in document.page_content:
        count += 1

if count > 1:
    print("Test string found multiple times in documents.")
else:
    print("Test string not found multiple times in documents.")
print(count)

Test string not found multiple times in documents.
1


RecursiveChar splitter makes 187 chunks where as token splitter does it in 78 chunks


Using **RecursiveCharacterTextSplitter**

In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# r_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1024,
#     chunk_overlap=64,
# )
# count = 0;
# chunks = r_splitter.split_documents(documents)
# for chunk in chunks:
#   count+=1
#   print(chunk.page_content, chunk.metadata, '\n\n\n')
# print(count)

Bitcoin: A Peer-to-Peer Electronic Cash System
Satoshi Nakamoto
satoshin@gmx.com
www.bitcoin.org
Abstract.  A purely peer-to-peer version of electronic cash would allow online 
payments to be sent directly from one party to another without going through a 
financial institution.  Digital signatures provide part of the solution, but the main 
benefits are lost if a trusted third party is still required to prevent double-spending. 
We propose a solution to the double-spending problem using a peer-to-peer network. 
The network timestamps transactions by hashing them into an ongoing chain of 
hash-based proof-of-work, forming a record that cannot be changed without redoing 
the proof-of-work.  The longest chain not only serves as proof of the sequence of 
events witnessed, but proof that it came from the largest pool of CPU power.  As 
long as a majority of CPU power is controlled by nodes that are not cooperating to 
attack the network, they'll generate the longest chain and outpace attac

Using **TokenTextSplitter**

In [ ]:
from langchain.text_splitter import TokenTextSplitter

t_splitter = TokenTextSplitter(
    chunk_size=1024,
    chunk_overlap=64,
)
count = 0;
chunks = t_splitter.split_documents(documents)
for chunk in chunks:
  count+=1
  print(chunk.page_content, chunk.metadata, '\n\n\n')
print(count)

Bitcoin: A Peer-to-Peer Electronic Cash System
Satoshi Nakamoto
satoshin@gmx.com
www.bitcoin.org
Abstract.  A purely peer-to-peer version of electronic cash would allow online 
payments to be sent directly from one party to another without going through a 
financial institution.  Digital signatures provide part of the solution, but the main 
benefits are lost if a trusted third party is still required to prevent double-spending. 
We propose a solution to the double-spending problem using a peer-to-peer network. 
The network timestamps transactions by hashing them into an ongoing chain of 
hash-based proof-of-work, forming a record that cannot be changed without redoing 
the proof-of-work.  The longest chain not only serves as proof of the sequence of 
events witnessed, but proof that it came from the largest pool of CPU power.  As 
long as a majority of CPU power is controlled by nodes that are not cooperating to 
attack the network, they'll generate the longest chain and outpace attac

Need to convert the document datatype into string/raw text, as the models available on huggingface do not accpet document type.

In [ ]:
# fais_db = FAISS.from_texts(chunks,embeddings)

In [ ]:
# prompt: create a code to convert the datatype of chunks which is documents currently into text

# text_chunks = []
# for chunk in chunks:
#   text_chunks.append(chunk.page_content)

In [ ]:
# embeddings = model.encode(text_chunks)

In [ ]:
# fais_db = FAISS.from_texts(text_chunks,embeddings)

**TODO:** add metadata & fais_db local variable make it save in the memory,
store the vector database as a file in the hardisk

**OR** we can use FAISS.from_documents(docs,embeddings) as well, this directly takes the documents instead of the text and use embeddings on these docs.

In [ ]:
# faiss_db = FAISS.from_documents(chunks)

AttributeError: 'tuple' object has no attribute 'page_content'

# Using faiss.add_documents

In [ ]:
# from langchain.docstore.in_memory import InMemoryDocstore
# example_embedding = embeddings.embed_documents(["test"])[0]
# dimension = len(example_embedding)

# # Initialize an empty FAISS index
# index = faiss.IndexFlatL2(dimension)  # Create a FAISS index

#     # Initialize the FAISS vector store with required parameters
# faiss_db = FAISS(
#         embedding_function=embeddings.embed_documents,
#         index=index,
#         docstore=InMemoryDocstore({}),
#         index_to_docstore_id={}
#     )

#     # Add documents to the FAISS vector store
# faiss_db.add_documents(chunks,dtype = 'object')

1.   The issue arises when the ***add_documents***  method in FAISS does not batch API calls, which can result in slow performance when adding a large number of documents. A more efficient approach is to use the ***embed_documents***  method to get the embeddings in batches and then use the add_embeddings method to add them to the FAISS index.
2.   So instead of doing one chunk at a time and making api call of embedding for that one chunk then adding it to the vector store one by one, we make a batch of multiple chunks and make a single api call per batch and do this for all the batches first and then add all the embeddings to the vector store in one go
3.    The batch processing approach optimizes the workflow by reducing the overhead of multiple API calls and individual insertions into the vector store.
4.    Also, an error is thrown: *setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (78,) + inhomogeneous part.*

# Using faiss.add_embeddings with Batches

In [ ]:
from langchain.docstore.in_memory import InMemoryDocstore
example_embedding = embeddings.embed_documents(["test"])[0]
dimension = len(example_embedding)

# Initialize an empty FAISS index
index = faiss.IndexFlatL2(dimension)

# Initialize the FAISS vector store with required parameters
faiss_db = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore({}),
    index_to_docstore_id={}
)

# Batch size
batch_size = 39

# Function to embed documents in batches and include metadata
def embed_documents_in_batches_with_metadata(docs, batch_size):
    all_embeddings = []
    all_metadatas = []
    for i in range(0, len(docs), batch_size):
        batch_docs = docs[i:i+batch_size]
        batch_texts = [doc.page_content for doc in docs]
        batch_embeddings = embeddings.embed_documents([doc.page_content for doc in batch_docs])
        all_embeddings.extend(batch_embeddings)
        all_metadatas.extend([doc.metadata for doc in batch_docs])
    return all_embeddings, all_metadatas, batch_texts

# Embed documents in batches
all_embeddings, all_metadatas, batch_texts = embed_documents_in_batches_with_metadata(chunks, batch_size)

In [ ]:
# Add all embeddings and metadata to the FAISS vector store
faiss_db.add_embeddings(text_embeddings=zip(batch_texts, all_embeddings), metadatas=all_metadatas) # Pass in a list of tuples, where each tuple is (text, embedding)

['8b1c76d9-a02a-4a67-ab63-17c5f5368767',
 'febd4c3e-788c-4c10-a821-609e4994eb62',
 '323dcbab-b46d-46ea-861c-0c7259b17c66',
 'a3f1cd3d-6bc9-4f96-bd54-6b3cc53c05d7',
 '10f6d042-3d0b-4185-bc08-ee6223c25cf9',
 '6bee976e-27d7-47c6-9c78-7d7752efb809',
 'ec184cfa-6cb3-424e-b9b2-b78646dd4b8b',
 '8497e391-92b3-49a8-83f4-6d60990dc2fd',
 '230de216-a837-44dd-9633-0a9eaf1d168f',
 '3bd2fdb9-7f46-4546-8dad-b5a1ce326011',
 '5c4cd186-42a7-40b6-afe4-049efcbfde9d',
 'ce38ae4a-057d-4279-a80f-aace971f2a47',
 '0722aedf-a21d-4300-b24c-ae197a5c85cd',
 'e1b9e076-3ef2-4b43-87b8-634f26ad6bad',
 '1726d542-847f-4947-9539-a65692d0961a',
 '4130c61e-455e-4859-a903-02518be20a13',
 'd1f36550-9c8f-421c-817e-51f5100a8fb5',
 'e1424f81-2897-4658-a5d7-5bc428143358',
 '78054bb0-f1d7-4943-a686-96f2cb213aaf',
 '4202050f-d9b2-4a2e-974b-f8d34ce6ac94',
 '22224069-9360-4046-8478-512224af1e6b',
 '7f7ac68d-a98d-40db-b491-63fe72dce9f1',
 '05ad0639-6619-451d-a7b1-a31b9e4585e9',
 'eb2d5cb0-5220-46ee-94e3-29c53d308972',
 '7dba0f68-6ef7-

In [ ]:
print(faiss_db.index.ntotal)

78


In [ ]:
query = 'what is bitcoin?'
docs = faiss_db.similarity_search(query)
print(docs)

[Document(page_content="Bitcoin: A Peer-to-Peer Electronic Cash System\nSatoshi Nakamoto\nsatoshin@gmx.com\nwww.bitcoin.org\nAbstract.  A purely peer-to-peer version of electronic cash would allow online \npayments to be sent directly from one party to another without going through a \nfinancial institution.  Digital signatures provide part of the solution, but the main \nbenefits are lost if a trusted third party is still required to prevent double-spending. \nWe propose a solution to the double-spending problem using a peer-to-peer network. \nThe network timestamps transactions by hashing them into an ongoing chain of \nhash-based proof-of-work, forming a record that cannot be changed without redoing \nthe proof-of-work.  The longest chain not only serves as proof of the sequence of \nevents witnessed, but proof that it came from the largest pool of CPU power.  As \nlong as a majority of CPU power is controlled by nodes that are not cooperating to \nattack the network, they'll genera

**Storing the vectorstore** on local stoarge and then donwload on the hard disk

In [ ]:
# Define the path where you want to save the FAISS index and metadata
save_path = '/content/faiss_save'

# Save the FAISS index and metadata
faiss_db.save_local(save_path)

print(f"FAISS index and metadata saved to {save_path}")

FAISS index and metadata saved to /content/faiss_save


# Retriever methods

---



In [ ]:
from langchain.retrievers import LongContextReorder
from langchain.retrievers import EnsembleRetriever
# def create_retriever(retriever_choice):
#     if retriever_choice == 'long_context_retriever':
#         from langchain_community.document_transformers import (LongContextReorder)
#         retriever = LongContextReorder(vector_store=faiss_db, embedding_function=embeddings.embed_query)
#     elif retriever_choice == 'ensemble_retriever':
#         from langchain.retrievers import VectorStoreRetriever
#         retriever = VectorStoreRetriever(vector_store=faiss_db)
#     else:
#         raise ValueError("Invalid retriever choice. Choose 'self' or 'vectorstore'.")
#     return retriever

# Everything into a single function with folder path as input

In [ ]:
import os
import json
import faiss
import shutil
from langchain.document_loaders import PyMuPDFLoader, TextLoader ,CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter,  TokenTextSplitter, CharacterTextSplitter
from langchain.docstore.in_memory import InMemoryDocstore
from langchain_community.document_transformers import LongContextReorder
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

# Load configurations from provided config_path
def load_configuration(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    return config

# Initialize a save function to save the vectorstore to desired save_path
def save_faiss_index(faiss_db, save_path):
    faiss_db.save_local(save_path)
    print(f"FAISS index and metadata saved to {save_path}")

# Initialize a load function to load the vectorstore from desired save_path
def load_faiss_index(save_path, embeddings):
    index_file = os.path.join(save_path, "index.faiss")
    docstore_file = os.path.join(save_path, "index.pkl")

    if os.path.exists(index_file) and os.path.exists(docstore_file):
        faiss_db = FAISS.load_local(save_path, embeddings, allow_dangerous_deserialization = True)
        print(f"FAISS index and metadata loaded from {save_path}")
        return faiss_db
    else:
        print(f"No saved FAISS index found at {save_path}. Creating a new one.")
        return None

def multidoc_vectorstore(config):

  # Extract parameters from config.json
  embedding_model_name = config['embedding_model']
  source_folder = config['ingestion']['source_folder']
  processed_folder = config['ingestion']['processed_folder']
  error_folder = config['ingestion']['error_folder']
  chunk_size = config['ingestion']['chunk_size']
  chunk_overlap = config['ingestion']['chunk_overlap']
  save_path = config['ingestion']['save_path']
  splitter_method = config['ingestion']['splitter']
  retriever_choice = config['retriever']
  top_k = config['top_k']
  score = config['score']

  faiss_db = None  # Declare faiss_db
  documents = []

  # Initialize the HuggingFaceEmbeddings object
  embeddings = HuggingFaceEmbeddings(
      model_name=embedding_model_name,
      model_kwargs={"device": "cpu"},
      encode_kwargs={"normalize_embeddings": True},
  )

  # Try to load existing FAISS index
  faiss_db = load_faiss_index(save_path, embeddings)
  if not faiss_db:

      dimension = embeddings.client.get_sentence_embedding_dimension()
      # Initialize an empty FAISS index
      index = faiss.IndexFlatL2(dimension)

      # Initialize the FAISS vector store with required parameters
      faiss_db = FAISS(
          embedding_function=embeddings,
          index=index,
          docstore=InMemoryDocstore({}),
          index_to_docstore_id={}
      )

  # Specify the folder path to scan the documents
  for filename in os.listdir(source_folder):
      file_path = os.path.join(source_folder, filename)

      if filename.endswith(('.pdf', '.txt', '.csv')):
          try:
            if filename.endswith('.pdf'):
                loader = PyMuPDFLoader(file_path)
                loaded_docs = loader.load()

                for doc in loaded_docs:
                    # Add metadata
                    doc.metadata['source'] = filename
                documents.extend(loaded_docs)

            elif filename.endswith('.txt'):
                loader = TextLoader(file_path)
                loaded_docs = loader.load()

                for doc in loaded_docs:
                    # Add metadata
                    doc.metadata['source'] = filename
                documents.extend(loaded_docs)

            elif filename.endswith('.csv'):
                loader = CSVLoader(file_path)
                loaded_docs = loader.load()

                for doc in loaded_docs:
                    # Add metadata
                    doc.metadata['source'] = filename
                documents.extend(loaded_docs)

          except Exception as e:
            print(f"Error loading file {filename}: {e}")
            error_path = os.path.join(error_folder, filename)
            shutil.move(file_path, error_path)
            continue

      else:
          # Move unsupported file types to the error folder
          error_path = os.path.join(error_folder, filename)
          shutil.move(file_path,error_path)
          return "Unsupported file type"

 # Choose which splitting method to use
  if splitter_method == 'recursive':
      splitter = RecursiveCharacterTextSplitter(
          chunk_size = chunk_size,
          chunk_overlap = chunk_overlap,
      )
  elif splitter_method == 'token':
      splitter = TokenTextSplitter(
          chunk_size = chunk_size,
          chunk_overlap = chunk_overlap,
      )
  elif splitter_method == 'character':
      splitter = CharacterTextSplitter(
          chunk_size = chunk_size,
          chunk_overlap = chunk_overlap,
      )
  else:
      raise ValueError("Invalid text_splitter_method. Choose from 'recursive', 'token', 'character'.")

  chunks = splitter.split_documents(documents)


  # Batch size
  batch_size = 39

  # Function to embed documents in batches and include metadata
  def embed_documents_in_batches_with_metadata(docs, batch_size):
      all_embeddings = []
      all_metadatas = []
      all_texts = []
      for i in range(0, len(docs), batch_size):
          batch_docs = docs[i:i+batch_size]
          batch_texts = [doc.page_content for doc in batch_docs]
          batch_embeddings = embeddings.embed_documents([doc.page_content for doc in batch_docs])
          all_embeddings.extend(batch_embeddings)
          all_metadatas.extend([doc.metadata for doc in batch_docs])
          all_texts.extend(batch_texts)
      return all_embeddings, all_metadatas, all_texts

  try:
    # Embed documents in batches
    all_embeddings, all_metadatas, all_texts = embed_documents_in_batches_with_metadata(chunks, batch_size)

    # Add all embeddings and metadata to the FAISS vector store
    faiss_db.add_embeddings(
        text_embeddings=zip(all_texts, all_embeddings),
        metadatas=all_metadatas
        ) # Pass in a list of tuples, where each tuple is (text, embedding)

  except Exception as e:
    print(f"Error embedding documents: {e}")
    for doc in chunks:
        error_path = os.path.join(error_folder, doc.metadata['source'])
        shutil.move(os.path.join(source_folder, doc.metadata['source']), error_path)

  # Save the FAISS vector store
  save_faiss_index(faiss_db, save_path)

  # Move processed files to the processed folder
  for filename in os.listdir(source_folder):
      file_path = os.path.join(source_folder, filename)
      processed_path = os.path.join(processed_folder, filename)
      shutil.move(file_path, processed_path)

  return faiss_db, chunks, top_k, score

def create_retrievers(faiss_db, retriever_choice: str, chunks):
    if retriever_choice == "long_context_reorder":
        retriever = LongContextReorder(retriever=faiss_db.as_retriever())  # Convert FAISS to retriever
    elif retriever_choice == "ensemble":
        # Pass in chunks as a keyword argument
        bm25_retriever = BM25Retriever.from_documents(chunks)
        # bm25_retriever.k = top_k
        default_retriever = faiss_db.as_retriever()  # Convert FAISS to retriever
        retriever = EnsembleRetriever(
            retrievers=[default_retriever, bm25_retriever],  # Pass the retriever objects directly
            aggregation_method="sum"  # You can choose "sum" or "average"
        )
    elif retriever_choice == "default":
        retriever = faiss_db.as_retriever()  # Convert FAISS to retriever
    else:
        raise ValueError("Invalid retriever method. Choose from 'long_context_reorder', 'ensemble', 'default'")
    return retriever

# Function to limit the number of results
def limit_results(results, top_k):
    return results[:top_k]

def main(config_path):
    config = load_configuration(config_path)
    faiss_db, chunks, top_k, score = multidoc_vectorstore(config)
    retriever = create_retrievers(faiss_db, config['retriever'], chunks)
    return faiss_db, retriever, top_k, score

# Example usage
config_path = '/content/config.json'
faiss_db, retriever, top_k, score = main(config_path)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

No saved FAISS index found at /content/faiss_save. Creating a new one.
FAISS index and metadata saved to /content/faiss_save


# Testing


In [ ]:
import os
import json
import faiss
import shutil
from langchain.document_loaders import PyMuPDFLoader, TextLoader ,CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter,  TokenTextSplitter, CharacterTextSplitter
from langchain.docstore.in_memory import InMemoryDocstore
from langchain_community.document_transformers import LongContextReorder
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

# Load configurations from provided config_path
def load_configuration(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    return config

# Initialize a save function to save the vectorstore to desired save_path
def save_faiss_index(faiss_db, save_path):
    faiss_db.save_local(save_path)
    print(f"FAISS index and metadata saved to {save_path}")

# Initialize a load function to load the vectorstore from desired save_path
def load_faiss_index(save_path, embeddings):
    index_file = os.path.join(save_path, "index.faiss")
    docstore_file = os.path.join(save_path, "index.pkl")

    if os.path.exists(index_file) and os.path.exists(docstore_file):
        faiss_db = FAISS.load_local(save_path, embeddings, allow_dangerous_deserialization = True)
        print(f"FAISS index and metadata loaded from {save_path}")
        return faiss_db
    else:
        print(f"No saved FAISS index found at {save_path}. Creating a new one.")
        return None

def multidoc_vectorstore(config):

  # Extract parameters from config.json
  embedding_model_name = config['embedding_model']
  source_folder = config['ingestion']['source_folder']
  processed_folder = config['ingestion']['processed_folder']
  error_folder = config['ingestion']['error_folder']
  chunk_size = config['ingestion']['chunk_size']
  chunk_overlap = config['ingestion']['chunk_overlap']
  save_path = config['ingestion']['save_path']
  splitter_method = config['ingestion']['splitter']
  retriever_choice = config['retriever']
  top_k = config['top_k']
  score = config['score']

  faiss_db = None  # Declare faiss_db
  documents = []

  # Initialize the HuggingFaceEmbeddings object
  embeddings = HuggingFaceEmbeddings(
      model_name=embedding_model_name,
      model_kwargs={"device": "cpu"},
      encode_kwargs={"normalize_embeddings": True},
  )

  # Try to load existing FAISS index
  faiss_db = load_faiss_index(save_path, embeddings)
  if not faiss_db:

      dimension = embeddings.client.get_sentence_embedding_dimension()
      # Initialize an empty FAISS index
      index = faiss.IndexFlatL2(dimension)

      # Initialize the FAISS vector store with required parameters
      faiss_db = FAISS(
          embedding_function=embeddings,
          index=index,
          docstore=InMemoryDocstore({}),
          index_to_docstore_id={}
      )

  # Specify the folder path to scan the documents
  for filename in os.listdir(source_folder):
      file_path = os.path.join(source_folder, filename)

      if filename.endswith(('.pdf', '.txt', '.csv')):
          try:
            if filename.endswith('.pdf'):
                loader = PyMuPDFLoader(file_path)
                loaded_docs = loader.load()

                for doc in loaded_docs:
                    # Add metadata
                    doc.metadata['source'] = filename
                documents.extend(loaded_docs)

            elif filename.endswith('.txt'):
                loader = TextLoader(file_path)
                loaded_docs = loader.load()

                for doc in loaded_docs:
                    # Add metadata
                    doc.metadata['source'] = filename
                documents.extend(loaded_docs)

            elif filename.endswith('.csv'):
                loader = CSVLoader(file_path)
                loaded_docs = loader.load()

                for doc in loaded_docs:
                    # Add metadata
                    doc.metadata['source'] = filename
                documents.extend(loaded_docs)

          except Exception as e:
            print(f"Error loading file {filename}: {e}")
            error_path = os.path.join(error_folder, filename)
            shutil.move(file_path, error_path)
            continue

      else:
          # Move unsupported file types to the error folder
          error_path = os.path.join(error_folder, filename)
          shutil.move(file_path,error_path)
          print("Unsupported file type")
          return None, None, None, None

 # Choose which splitting method to use
  if splitter_method == 'recursive':
      splitter = RecursiveCharacterTextSplitter(
          chunk_size = chunk_size,
          chunk_overlap = chunk_overlap,
      )
  elif splitter_method == 'token':
      splitter = TokenTextSplitter(
          chunk_size = chunk_size,
          chunk_overlap = chunk_overlap,
      )
  elif splitter_method == 'character':
      splitter = CharacterTextSplitter(
          chunk_size = chunk_size,
          chunk_overlap = chunk_overlap,
      )
  else:
      raise ValueError("Invalid text_splitter_method. Choose from 'recursive', 'token', 'character'.")

  chunks = splitter.split_documents(documents)


  # Batch size
  batch_size = 39

  # Function to embed documents in batches and include metadata
  def embed_documents_in_batches_with_metadata(docs, batch_size):
      all_embeddings = []
      all_metadatas = []
      all_texts = []
      for i in range(0, len(docs), batch_size):
          batch_docs = docs[i:i+batch_size]
          batch_texts = [doc.page_content for doc in batch_docs]
          batch_embeddings = embeddings.embed_documents([doc.page_content for doc in batch_docs])
          all_embeddings.extend(batch_embeddings)
          all_metadatas.extend([doc.metadata for doc in batch_docs])
          all_texts.extend(batch_texts)
      return all_embeddings, all_metadatas, all_texts

  try:
    # Embed documents in batches
    all_embeddings, all_metadatas, all_texts = embed_documents_in_batches_with_metadata(chunks, batch_size)

    # Add all embeddings and metadata to the FAISS vector store
    faiss_db.add_embeddings(
        text_embeddings=zip(all_texts, all_embeddings),
        metadatas=all_metadatas
        ) # Pass in a list of tuples, where each tuple is (text, embedding)

  except Exception as e:
    print(f"Error embedding documents: {e}")
    for doc in chunks:
        error_path = os.path.join(error_folder, doc.metadata['source'])
        shutil.move(os.path.join(source_folder, doc.metadata['source']), error_path)

  # Save the FAISS vector store
  save_faiss_index(faiss_db, save_path)

  # Move processed files to the processed folder
  for filename in os.listdir(source_folder):
      file_path = os.path.join(source_folder, filename)
      processed_path = os.path.join(processed_folder, filename)
      shutil.move(file_path, processed_path)

  return faiss_db, chunks, top_k, score

def create_retrievers(faiss_db, retriever_choice: str, chunks):
    if not faiss_db or chunks:
        print("FAISS index or chunks not initialized.")
        return None
    elif retriever_choice == "long_context_reorder":
        retriever = LongContextReorder(retriever=faiss_db.as_retriever())  # Convert FAISS to retriever
    elif retriever_choice == "ensemble":
        # Pass in chunks as a keyword argument
        bm25_retriever = BM25Retriever.from_documents(chunks)
        # bm25_retriever.k = top_k
        default_retriever = faiss_db.as_retriever()  # Convert FAISS to retriever
        retriever = EnsembleRetriever(
            retrievers=[default_retriever, bm25_retriever],  # Pass the retriever objects directly
            aggregation_method="sum"  # You can choose "sum" or "average"
        )
    elif retriever_choice == "default":
        retriever = faiss_db.as_retriever()  # Convert FAISS to retriever
    else:
        raise ValueError("Invalid retriever method. Choose from 'long_context_reorder', 'ensemble', 'default'")
    return retriever

# Function to limit the number of results
def limit_results(results, top_k):
    return results[:top_k]

def main(config_path):
    config = load_configuration(config_path)
    faiss_db, chunks, top_k, score = multidoc_vectorstore(config)
    retriever = create_retrievers(faiss_db, config['retriever'], chunks)
    return faiss_db, retriever, top_k, score

# Example usage
config_path = '/content/config.json'
faiss_db, retriever, top_k, score = main(config_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


FAISS index and metadata loaded from /content/faiss_save
Unsupported file type
FAISS index or chunks not initialized.


# Examples

In [ ]:
query = 'who is Luke?'
docs = retriever.invoke(query)
limited_docs = limit_results(docs, top_k)  # Limit the results to top_k

i = 0
for doc in limited_docs:
    i += 1
    print(doc)

print(i)

page_content='Luke Skywalker
Star Wars character
Mark Hamill as Luke Skywalker in
Return of the Jedi (top) and The
Last Jedi
First
appearance
Star Wars (1977)
Created by
George Lucas
Portrayed by
Mark Hamill
Grant Feely
Aidan Barton
Voiced by
Various [a]
In-universe information
Full name
Luke Skywalker
Species
Human
Gender
Male
Occupation
Moisture farmer
Jedi Apprentice
Jedi Knight
Affiliation
Rebel Alliance
Jedi Order
New Republic
New Jedi Order
Resistance
Luke Skywalker
Luke Skywalker is a character in the Star Wars franchise created
by George Lucas. He is the protagonist of the original film trilogy,
and also appears in the sequel trilogy. Raised as a farmer on the
fictional planet Tatooine, Luke joins the Rebel Alliance and
becomes a pivotal figure in the struggle against the Galactic
Empire. He trains as a Jedi under Obi-Wan Kenobi and Yoda, and
eventually confronts his father, the Sith Lord Darth Vader. Years
later, Luke trains his nephew Ben Solo and mentors the scavenger
Rey. L

In [ ]:
query = 'what is the feat_description of toss_decision_01?'
docs = retriever.invoke(query)
limited_docs = limit_results(docs, top_k)  # Limit the results to top_k

i = 0
for doc in limited_docs:
    i += 1
    print(doc)

print(i)

page_content='feat_id: 1\nfeat_name: toss_decision_01\nmodel_feat_imp_train: 0.06473549\nfeat_rank_train: 1\nfeat_description: Toss decision - categorical - 1 if winner bats, 0 otherwise' metadata={'source': 'file2.csv', 'row': 0}
page_content='feat_id: 2\nfeat_name: team_count_50runs_last15\nmodel_feat_imp_train: 0.056898594\nfeat_rank_train: 2\nfeat_description: Ratio of number of 50s by players in team1 to number of 50s by players in team2 in last 15 games' metadata={'source': 'file2.csv', 'row': 1}
page_content='feat_id: 10\nfeat_name: team2_bowling_econ_last10\nmodel_feat_imp_train: 0.04628582\nfeat_rank_train: 10\nfeat_description: No description available' metadata={'source': 'file2.csv', 'row': 9}
page_content='feat_id: 8\nfeat_name: team2_id\nmodel_feat_imp_train: 0.049476445\nfeat_rank_train: 8\nfeat_description: id of team2' metadata={'source': 'file2.csv', 'row': 7}
page_content='feat_id: 9\nfeat_name: team1_bowling_econ_last10\nmodel_feat_imp_train: 0.04663536\nfeat_rank_t

**TODO**

1.    Chunk size, chunk overlap, vector store path; path where the downloaded vector store is present, use that vector store, embedding. ✅
2.    Replace from docs with add docs. ✅
3.    Save in the sense, we make the embeddings of the context and then we store it in the vector database/store, and when we restart it, all the process of embedding the context and saving it in the store will be required to redo. So instead of redoing it everytime we need to save the vector store which has the embeddings done in the hardisk.✅




In [ ]:
query = 'what is bitcoin?'
docs = faiss_db.similarity_search_with_score(query, k=top_k, score=score)
print(docs)

[(Document(page_content="Bitcoin: A Peer-to-Peer Electronic Cash System\nSatoshi Nakamoto\nsatoshin@gmx.com\nwww.bitcoin.org\nAbstract.  A purely peer-to-peer version of electronic cash would allow online \npayments to be sent directly from one party to another without going through a \nfinancial institution.  Digital signatures provide part of the solution, but the main \nbenefits are lost if a trusted third party is still required to prevent double-spending. \nWe propose a solution to the double-spending problem using a peer-to-peer network. \nThe network timestamps transactions by hashing them into an ongoing chain of \nhash-based proof-of-work, forming a record that cannot be changed without redoing \nthe proof-of-work.  The longest chain not only serves as proof of the sequence of \nevents witnessed, but proof that it came from the largest pool of CPU power.  As \nlong as a majority of CPU power is controlled by nodes that are not cooperating to \nattack the network, they'll gener

In [ ]:
query = 'what is the feat_description of toss_decision_01?'
docs = faiss_db.similarity_search_with_score(query, k=top_k, score=score)
print(docs)

[(Document(page_content='feat_id: 1\nfeat_name: toss_decision_01\nmodel_feat_imp_train: 0.06473549\nfeat_rank_train: 1\nfeat_description: Toss decision - categorical - 1 if winner bats, 0 otherwise', metadata={'source': 'file2.csv', 'row': 0}), 0.13799952), (Document(page_content='feat_id: 10\nfeat_name: team2_bowling_econ_last10\nmodel_feat_imp_train: 0.04628582\nfeat_rank_train: 10\nfeat_description: No description available', metadata={'source': 'file2.csv', 'row': 9}), 0.22513196), (Document(page_content='feat_id: 9\nfeat_name: team1_bowling_econ_last10\nmodel_feat_imp_train: 0.04663536\nfeat_rank_train: 9\nfeat_description: No description available', metadata={'source': 'file2.csv', 'row': 8}), 0.23286498), (Document(page_content='feat_id: 5\nfeat_name: team1_strike_rate_last10_games\nmodel_feat_imp_train: 0.0523683\nfeat_rank_train: 5\nfeat_description: No description available', metadata={'source': 'file2.csv', 'row': 4}), 0.23517245), (Document(page_content='feat_id: 3\nfeat_na


In similarity search using FAISS, the ***similarity_search_with_score*** method typically returns a list of documents along with their similarity scores. The interpretation of these scores depends on the distance metric used in the FAISS index.


1.   ***L2 (Euclidean) Distance:*** Smaller scores indicate more similar documents.
2.  ***Inner Product (Dot Product):*** Higher scores indicate more similar documents when vectors are normalized.
3. ***Cosine Similarity:*** Similar to inner product but normalized; higher scores indicate more similar documents.
*So,* the interpretation of "high score" depends on the distance metric used:
1. L2 Distance: A high score means the document is less similar to the query, thus it will have a higher rank (less relevant).
2. Inner Product: A high score means the document is more similar to the query, thus it will have a lower rank (more relevant).

In [ ]:
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# Confirm that the 4 relevant documents are at beginning and end.
reordered_docs

[(Document(page_content='age, Hugh (April 4, 2018). "Mark Hamill says Luke Skywalker was used as a "plot\ndevice" in Star Wars: The Last Jedi" (http://www.digitalspy.com/movies/star-wars/news/a853\n926/mark-hamill-star-wars-the-last-jedi-criticisms-luke-skywalker-plot-device/). Digital Spy.\nArchived (https://web.archive.org/web/20181122052157/http://www.digitalspy.com/movies/st\nar-wars/news/a853926/mark-hamill-star-wars-the-last-jedi-criticisms-luke-skywalker-plot-dev\nice/) from the original on November 22, 2018. Retrieved June 5, 2018.\n37. Kamp, David (May 25, 2017). "Star Wars Nerds, Mark Hamill Is One of You" (https://www.van\nityfair.com/hollywood/2017/05/mark-hamill-star-wars-nerd). Vanity Fair. Archived (https://web.\narchive.org/web/20210808091948/https://www.vanityfair.com/hollywood/2017/05/mark-hamil\nl-star-wars-nerd) from the original on August 8, 2021. Retrieved June 5, 2018.\nRinzler, J.W. (2007). The Making of Star Wars: The Definitive Story Behind the Original Film\n

In [ ]:
i=0
for doc in docs:
  i+=1
  print(doc)

print(i)

(Document(page_content='feat_id: 1\nfeat_name: toss_decision_01\nmodel_feat_imp_train: 0.06473549\nfeat_rank_train: 1\nfeat_description: Toss decision - categorical - 1 if winner bats, 0 otherwise', metadata={'source': 'file2.csv', 'row': 0}), 0.13799952)
(Document(page_content='feat_id: 10\nfeat_name: team2_bowling_econ_last10\nmodel_feat_imp_train: 0.04628582\nfeat_rank_train: 10\nfeat_description: No description available', metadata={'source': 'file2.csv', 'row': 9}), 0.22513196)
(Document(page_content='feat_id: 9\nfeat_name: team1_bowling_econ_last10\nmodel_feat_imp_train: 0.04663536\nfeat_rank_train: 9\nfeat_description: No description available', metadata={'source': 'file2.csv', 'row': 8}), 0.23286498)
(Document(page_content='feat_id: 5\nfeat_name: team1_strike_rate_last10_games\nmodel_feat_imp_train: 0.0523683\nfeat_rank_train: 5\nfeat_description: No description available', metadata={'source': 'file2.csv', 'row': 4}), 0.23517245)
(Document(page_content='feat_id: 3\nfeat_name: t

In [ ]:
print(faiss_db.index.ntotal)

78


**TODO**

1. Check why there are 2 same results in the top_k results. ✅
2. Add Ensemble Retriever and Long-Context Reorder Retriever. ✅
3. Make a python script of all this functions.
4. Make a google drive link where the config and python script file will be uploaded and share it.
5. Use streamLit or Mesop (https://github.com/google/mesop) to design a webapp which takes query as input, has  a search button, and shows the metadata which contains type of doc and the page number from where the result is taken from. Then, when u click on the metadata page number, the pdf and the page should be visible on the side bar from where the result is taken from.
6. I think we should make another folder path for new files which would then get embedded and added on to the existing folder path, this makes sure that the older files are not embedded repeatedly with each new file, we can show it in the ui like: specify a spot to **add files** and specify another labelled **add more files.** ✅


**TODO**


1. Upload all of installed libraries in a .txt and specify version of everything thats installed ✅
2. Dont keep anything out of the function ✅
3. Dont make faiss_db global ✅
4. Fetch dimension size from sentence transformer, instead of passing a test string to get the dimension ✅
5. Make it true ✅
6. Manage the error_folder, we did manage it but we now have an error




In [ ]:
!pip freeze

absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.9.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.6.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.18.1
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.8
csvloader==0.0.2
cuda-python==12.2.1
cudf-cu12 @ http

# MESOP

In [ ]:
!pip install mesop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.6 MB/s eta 0:00:00


In [ ]:
import mesop as me
import mesop.labs as mel
me.colab_run()


Running server on: http://localhost:32123
 * Serving Flask app 'mesop.server.server'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (::)
 * Running on http://[::1]:32123
 * Running on http://[::1]:32123
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import random
import time

import mesop as me
import mesop.labs as mel

# Define your retriever function here or import from your existing module
def search_answer(query):
    results = retriever.invoke(query)
    limited_results = limit_results(results, 1)
    return limited_results

@me.page(path="/chat")
def chat():
    mel.chat(transform)

def transform(input, output_history) -> str:
    if isinstance(input, str):
        prompt = input.strip()  # If input is a string, directly use it as prompt
    elif hasattr(input, 'text'):
        prompt = input.text.strip()  # If input has 'text' attribute, use that as prompt
    else:
        prompt = str(input).strip()  # Fallback to converting input to string and stripping whitespace

    if prompt.endswith("?"):
        # If the prompt is a question, use the retriever function to get an answer
        answer_docs = search_answer(prompt)
        answer = ' '.join([doc.page_content for doc in answer_docs])
    else:
        # Otherwise, echo the prompt
        answer = f"You said: {prompt}, make sure the question ends with a question mark '?' "

    time.sleep(0.3)  # Simulate processing delay
    return answer

# Use colab_show to display the chat interface in Colab
me.colab_show(path="/chat")


<IPython.core.display.Javascript object>

In [ ]:
@me.page(path="/hello_world")
def chat():
  mel.chat(transform)

def transform(query):
  query

In [ ]:
me.colab_show(path="/hello_world", height=100)

<IPython.core.display.Javascript object>

In [ ]:
import mesop
print(dir(mesop.components))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'audio', 'badge', 'box', 'button', 'checkbox', 'code', 'divider', 'embed', 'html', 'icon', 'image', 'input', 'markdown', 'plot', 'progress_bar', 'progress_spinner', 'radio', 'select', 'sidenav', 'slide_toggle', 'slider', 'table', 'text', 'tooltip', 'uploader', 'video']
